Import the required packages and set up the Spark session. This would not be required if using DataBricks.

In [1]:
import findspark
findspark.init()
findspark.find()

'/home/graeme/anaconda3/envs/ENSF-612/lib/python3.9/site-packages/pyspark'

In [76]:
from pyspark.sql import SparkSession
import pyspark;
spark = SparkSession.builder.appName('A2').getOrCreate();

Note: We will be using Spark for GridSearchCV only. This appears to be the only sklearn method currently supported by Spark. The remaining code will be executed using Pandas dataframes.

Load the CSV file (n=1000 samples) containing our manual labels as the target vector

In [13]:
df = (spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .option("multiline", "true")
  .option("quote", '"')  
  .option("escape", "\\")
  .option("escape", '"')
  .load("GH-React.csv")
)

In [1]:
import pandas as pd

df = pd.read_csv("./GH-React.csv")

In [7]:
print("Shape:", df.shape)
df.columns

Shape: (1000, 16)


Index(['html_url', 'number', 'title', 'labels', 'state', 'locked', 'milestone',
       'comments', 'created_at', 'updated_at', 'closed_at',
       'author_association', 'state_reason', 'assignee.login', 'body',
       'Target'],
      dtype='object')

## Machine Learning Pipeline
### Stage 1
The preprocess() function is defined below. It takes in a String formatted as Markdown from GitHub and pre-processes it to return a new string ready for the next stages in our ML Pipeline.

In [13]:
import re

def preprocess(text):
    stripped = text.lower()

    # remove all headings, bold text, and HTML comments from the Markdown text.
    # These items have all been used by the React team in their issue templates on GitHub
    headings_pattern = r'(<=\s|^)#{1,6}(.*?)$'
    bold_pattern = r'\*\*(.+?)\*\*(?!\*)'
    comments_pattern = r'<!--((.|\n)*?)-->'
    combined_pattern = r'|'.join((headings_pattern, bold_pattern, comments_pattern))

    stripped = re.sub(combined_pattern, '', stripped)

    # find all URLs in the string, and then remove the final directory from each to leave the general URL form
    # there may be useful patterns based on what URLs issues are commonly linking to
    url_pattern = re.compile(r'(https?://[^\s]+)')
    for url in re.findall(url_pattern, stripped):
        new_url = url.rsplit("/", 1)[0]
        stripped = stripped.replace(url, new_url)
    
    non_alpha_pattern = r'[^A-Za-z\n ]+'
    stripped = re.sub(non_alpha_pattern, '', stripped)
    
    return stripped

In [11]:
test = df['body'][1]


In [14]:
preprocess(test)

'\n\n\nbug\n\n\na specific order of unmounting and remounting unstablecreatereturns from reactcallreturn causes an invariant violation in unmounthostcomponents\n\n\nthe following sandbox example crashes with an invariant violation when both the min and cycle props are odd numbers greater than zero\n\nhttpscodesandboxios\n\n\nthe app does not crash and cycles the number of items in the list\n\n\nreact and reactdom versions  and newer reactcallreturn version \n'

### Stage 2
Create a TF-IDF features matrix using TfidfVectorizer from sklearn applied to the title and body of each issue.

We will additionally add in the feature 'author_association' from the GitHub issue, as there may be a correlation between Members/Collaborators/Contributors submitting more valid bugs/feature requests than "None" users.

While lemmatization could have been done earlier in the pre-processsing stage, it is more efficient to lemmatize at this point in a custom_tokenizer() function passed to TfidfVectorizer since tokenization is part of both processses.

First, define the tokenizer and vectorizer:

In [ ]:
# Do not run yet. I'm not sure if we want to use this rexex tokenizer instead of built in lemma tokenizer

# Technicality: we want to use the regexp-based tokenizer
# that is used by CountVectorizer and only use the lemmatization
# from spacy. To this end, we replace en_nlp.tokenizer (the spacy tokenizer)
# with the regexp-based tokenization.
import re
# regexp used in CountVectorizer
regexp = re.compile('(?u)\\b\\w\\w+\\b')
# load spacy language model and save old tokenizer
en_nlp = spacy.load('en')
old_tokenizer = en_nlp.tokenizer
# replace the tokenizer with the preceding regexp
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(
regexp.findall(string))

In [30]:
%pip install spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.3 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [75]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import make_pipeline

nlp = spacy.load("en_core_web_sm")

# create a custom tokenizer using the spacy document processing pipeline
def custom_tokenizer(document):
    ppd = preprocess(document)
    doc = nlp(ppd)
    return [token.lemma_ for token in doc]

vect = TfidfVectorizer(tokenizer=custom_tokenizer, ngram_range=(1, 2))

Create the features matrix:

In [45]:
# combine the title and body into one column
text = (df['title'] + ' ' + df['body']).values.astype('U')

In [76]:
# create text_features using our vectorizer
text_features = vect.fit_transform(text)

In [77]:
from sklearn.preprocessing import OneHotEncoder

# use one hot encoder to transform the author_association to a feature set
ohe = OneHotEncoder()
author_association = ohe.fit_transform(df['author_association'].to_numpy().reshape(-1, 1))

In [78]:
from scipy.sparse import hstack

# create our final features matrix by combining both sets
X = hstack((text_features, author_association))

Create the target vector:

In [15]:
y = df['Target']

In [79]:
print("Shape of feature matrix:", X.shape)
print("Shape of target vector:", y.shape)

Shape of feature matrix: (1000, 75923)
Shape of target vector: (1000,)


### Stage 3
Split the data into training (80%) and validation(20%) sets. We will stratify based on the label since our dataset is imbalanced.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=1)

print(X_train.shape)
print(X_val.shape)

### Stage 4
Use cross-validate with our training set to test our model parameters

#### Grid Search for optimizing model

Use grid search to find potentially better model parameters:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

# Use spark_sklearn’s grid search instead:
from spark_sklearn import GridSearchCV

pipe = make_pipeline(TfidfVectorizer(tokenizer=custom_tokenizer), LogisticRegression())

param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
                "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)],
                "tfidfvectorizer__min_df": [0, 3, 5]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))

### Stage 5
Validate our chosen model against the validation set we saved in Stage 3